In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation

from PIL import Image
from PIL import ImageDraw




Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='viridis', magnitude= False, lw=15, pixels_per_node=31, x_name='X', y_name='Y', var_name=''):#, logvalues=False ):
    pixel_xscaling_factor = pixels_per_node/grid.dx
    pixel_yscaling_factor = pixels_per_node/grid.dy
    im = Image.new('RGBA', (round(grid.grid_xdimension*pixel_xscaling_factor), round(grid.grid_ydimension*pixel_yscaling_factor)), (255,255,255,255))
    draw = ImageDraw.Draw(im)

    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]       
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    #if logvalues:
    #    values = np.log10(abs(grid.at_link[value_name]))
    
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    for i, value in enumerate(values):
        draw.line( ((link_head_x[i]*pixel_xscaling_factor,link_head_y[i]*pixel_yscaling_factor),(link_tail_x[i]*pixel_xscaling_factor,link_tail_y[i]*pixel_yscaling_factor)),fill=scalarMap.to_rgba(value,bytes=True),width=lw) 
        #xs = [link_head_x[i],link_tail_x[i]]
        #ys = [link_head_y[i],link_tail_y[i]]
        #img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
    
    imshow(np.asarray(im), origin='lower', extent=(0,grid.grid_xdimension,0,grid.grid_ydimension))
    cb = colorbar(scalarMap)
    cb.ax.set_ylabel(var_name)
    xlabel(x_name)
    ylabel(y_name)


In [3]:
mg = RasterModelGrid((75,125),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
h = mg.add_zeros('node', 'hydraulic__head')
Q = mg.add_zeros('link', 'conduit__discharge')

#add input to node 
moulin_Q=1.
mg.at_node['input__discharge'][125*30+10]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
#Set boundary types
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
#Initialize flow in links (solver will update this, but it can't be zero)
Q[mg.active_links] = moulin_Q/mg.number_of_links
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print("Number of links = ", mg.number_of_links)
print("Number of nodes = ", mg.number_of_nodes)
print("Number of active links = ", mg.number_of_active_links)
print("Number of core nodes = ", mg.number_of_core_nodes)

#Initialize random hydraulic diameters
d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)

#Create flow solver on grid
pfn = PresFlowNetwork(mg)
#Run flow solver
pfn.run_one_step()
#Create melt-creep solver for conduit size evolution
mc = MeltCreep(mg, dt=1000.)


Number of links =  18550
Number of nodes =  9375
Number of active links =  17835
Number of core nodes =  8979


/home/mcoving/Data/github/landlab/landlab/components/conduit_networks/pressurized_flow.py:153: RuntimeWarning: divide by zero encountered in true_divide
  self.r = 8*f*L/(g*r_s**2.*d_h**5.)


Number of iterations = 1 tolerance = 0.403559942923
Number of iterations = 2 tolerance = 0.179620701685
Number of iterations = 3 tolerance = 0.041495783938
Number of iterations = 4 tolerance = 0.0162063695977
Number of iterations = 5 tolerance = 0.00585647423882
Number of iterations = 6 tolerance = 0.00245108835563
Number of iterations = 7 tolerance = 0.00202208402203
Number of iterations = 8 tolerance = 0.00207042492767
Number of iterations = 9 tolerance = 0.000825919679657
Number of iterations = 10 tolerance = 0.0181278240378
Number of iterations = 11 tolerance = 0.0142221296363
Number of iterations = 12 tolerance = 0.0538895317924
Number of iterations = 13 tolerance = 0.122097685508
Number of iterations = 14 tolerance = 0.254531137811
Number of iterations = 15 tolerance = 0.445792116605
Number of iterations = 16 tolerance = 1.88637004055
Number of iterations = 17 tolerance = 0.346864285775
Number of iterations = 18 tolerance = 0.160870608269
Number of iterations = 19 tolerance = 0.0

In [5]:
nsteps = 20000
step_start = 1
fig1, axs = subplots(2,2,figsize=(10,6))
every = 25
for step in arange(nsteps)+step_start:
    print("step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean())
    pfn.run_one_step()    
    #mg.calc_grad_at_link(h, out=dhdx)
    #h = fsolve(network_residuals, h[mg.core_nodes])
    #Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(2,2,1)
        imshow_grid_at_node(mg, h, var_name='Head ($m$)', cmap='viridis')#,vmin=300,vmax=400)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        #ylim([300,400])
        subplot(2,2,2)
        plot_links(mg, 'conduit__discharge', magnitude=True, var_name='Discharge ($m^3/s$)')#, vmin=0., vmax=1, autoscale=False)
        #plot(mg.at_link['conduit__discharge'][mg.active_links])
        #ylabel('')
        #ylim([0,1.5])
        subplot(2,2,3)
        plot_links(mg, 'hydraulic__diameter', var_name='Hydraulic diameter ($m$)')#, autoscale=False, vmin=0, vmax=1.25)
        subplot(2,2,4)#, position=[0.2,0.2,0.9,0.9])
        plot(mg.at_link['hydraulic__diameter'][mg.active_links])
        xlabel('Link number')
        ylabel('Hydraulic diameter ($m$)')
        ylim(0,1.25)
        #ylim([0.5,1.5])
        image_name = 'Big-example-grid/heads_and_discharge'+str(step).zfill(6)+'.png'
        #tight_layout()
        subplots_adjust(wspace=0.3)
        savefig(image_name,dpi=300)
        fig1.clf()


step = 1  avg d_h= 0.239502469299
Number of iterations = 1 tolerance = 0.000429396944206
Number of iterations = 2 tolerance = 0.000190073194535
Number of iterations = 3 tolerance = 7.03387491993e-05
Number of iterations = 4 tolerance = 1.44324860582e-05
Number of iterations = 5 tolerance = 7.30947455126e-07
max ddh =  0.00302386185638
step = 2  avg d_h= 0.239410074164
Number of iterations = 1 tolerance = 0.00251732849022
Number of iterations = 2 tolerance = 1.06183219952e-05
Number of iterations = 3 tolerance = 5.00290064186e-08
max ddh =  0.00290653572371
step = 3  avg d_h= 0.239317740152
Number of iterations = 1 tolerance = 0.00235345691866
Number of iterations = 2 tolerance = 8.53085917608e-06
max ddh =  0.00279704384002
step = 4  avg d_h= 0.239225471941
Number of iterations = 1 tolerance = 0.00220432828901
Number of iterations = 2 tolerance = 6.92190289656e-06
max ddh =  0.0026947139147
step = 5  avg d_h= 0.239133273669
Number of iterations = 1 tolerance = 0.00206834128483
Number o

Number of iterations = 1 tolerance = 0.00061747970395
Number of iterations = 2 tolerance = 1.03373484549e-06
max ddh =  0.00116085498412
step = 46  avg d_h= 0.235435156071
Number of iterations = 1 tolerance = 0.000606372697702
Number of iterations = 2 tolerance = 9.90553099468e-07
max ddh =  0.00114530789714
step = 47  avg d_h= 0.235347191516
Number of iterations = 1 tolerance = 0.000595627706813
Number of iterations = 2 tolerance = 1.03497802561e-06
max ddh =  0.00113018142126
step = 48  avg d_h= 0.235259338204
Number of iterations = 1 tolerance = 0.000585229027669
Number of iterations = 2 tolerance = 1.08365550219e-06
max ddh =  0.0011154586584
step = 49  avg d_h= 0.235171596275
Number of iterations = 1 tolerance = 0.000575161899779
Number of iterations = 2 tolerance = 1.13726368599e-06
max ddh =  0.00110112361096
step = 50  avg d_h= 0.235083965864
Number of iterations = 1 tolerance = 0.000565412374034
Number of iterations = 2 tolerance = 1.19660341578e-06
max ddh =  0.00108716112223

Number of iterations = 1 tolerance = 0.000330552745617
Number of iterations = 2 tolerance = 8.44800723396e-07
max ddh =  0.000718526637491
step = 92  avg d_h= 0.231505055888
Number of iterations = 1 tolerance = 0.000327135226466
Number of iterations = 2 tolerance = 8.03160840468e-07
max ddh =  0.000712703036357
step = 93  avg d_h= 0.231422267941
Number of iterations = 1 tolerance = 0.000323787423429
Number of iterations = 2 tolerance = 7.64721046187e-07
max ddh =  0.000706976170784
step = 94  avg d_h= 0.231339592555
Number of iterations = 1 tolerance = 0.000320508225503
Number of iterations = 2 tolerance = 7.2907385658e-07
max ddh =  0.00070134365119
step = 95  avg d_h= 0.231257029695
Number of iterations = 1 tolerance = 0.000317295491174
Number of iterations = 2 tolerance = 6.95986291152e-07
max ddh =  0.000695803165566
step = 96  avg d_h= 0.231174579325
Number of iterations = 1 tolerance = 0.000314147261293
Number of iterations = 2 tolerance = 1.68932180503e-06
max ddh =  0.000690352

Number of iterations = 2 tolerance = 6.97893744955e-07
max ddh =  0.000520958176312
step = 139  avg d_h= 0.227734915351
Number of iterations = 1 tolerance = 0.000221426160992
Number of iterations = 2 tolerance = 5.44667935868e-07
max ddh =  0.000517968316309
step = 140  avg d_h= 0.227657361138
Number of iterations = 1 tolerance = 0.000219960960701
Number of iterations = 2 tolerance = 4.39337684057e-07
max ddh =  0.000515014260341
step = 141  avg d_h= 0.227579916688
Number of iterations = 1 tolerance = 0.000218517988563
Number of iterations = 2 tolerance = 3.63355807573e-07
max ddh =  0.000512095374217
step = 142  avg d_h= 0.227502581923
Number of iterations = 1 tolerance = 0.000217096724475
Number of iterations = 2 tolerance = 3.06486050437e-07
max ddh =  0.000509211038662
step = 143  avg d_h= 0.227425356766
Number of iterations = 1 tolerance = 0.000215696701872
Number of iterations = 2 tolerance = 2.85623983846e-07
max ddh =  0.000506360648646
step = 144  avg d_h= 0.227348241136
Numbe

Number of iterations = 2 tolerance = 1.05526413424e-07
max ddh =  0.000410974537821
step = 186  avg d_h= 0.224207177137
Number of iterations = 1 tolerance = 0.000170579847602
Number of iterations = 2 tolerance = 1.04976663962e-07
max ddh =  0.000409163884379
step = 187  avg d_h= 0.224134690405
Number of iterations = 1 tolerance = 0.000169795520868
Number of iterations = 2 tolerance = 1.04431779845e-07
max ddh =  0.000407370276691
step = 188  avg d_h= 0.224062309319
Number of iterations = 1 tolerance = 0.000169020241101
Number of iterations = 2 tolerance = 1.03893217675e-07
max ddh =  0.000405593478948
step = 189  avg d_h= 0.223990033782
Number of iterations = 1 tolerance = 0.00016825385643
Number of iterations = 2 tolerance = 1.03356801971e-07
max ddh =  0.000403833259473
step = 190  avg d_h= 0.223917863699
Number of iterations = 1 tolerance = 0.000167496241968
Number of iterations = 2 tolerance = 1.02834215806e-07
max ddh =  0.000402089391415
step = 191  avg d_h= 0.223845798973
Number

...

step = 18026  avg d_h= 0.129511082642
Number of iterations = 1 tolerance = 1.50952526643e-05
Number of iterations = 2 tolerance = 4.72389092079e-09
max ddh =  2.34444529539e-05
step = 18027  avg d_h= 0.12950906078
Number of iterations = 1 tolerance = 1.50957792382e-05
Number of iterations = 2 tolerance = 1.26026102401e-09
max ddh =  2.34432616637e-05
step = 18028  avg d_h= 0.129507038988
Number of iterations = 1 tolerance = 1.50962305434e-05
Number of iterations = 2 tolerance = 1.24099393262e-09
max ddh =  2.34420703541e-05
step = 18029  avg d_h= 0.129505017265
Number of iterations = 1 tolerance = 1.50967283843e-05
Number of iterations = 2 tolerance = 1.23245483957e-09
max ddh =  2.34408790244e-05
step = 18030  avg d_h= 0.129502995612
Number of iterations = 1 tolerance = 1.50972249133e-05
Number of iterations = 2 tolerance = 1.21492590294e-09
max ddh =  2.34396878226e-05
step = 18031  avg d_h= 0.129500974029
Number of iterations = 1 tolerance = 1.50976848186e-05
Number of iterations = 

Number of iterations = 2 tolerance = 2.94533916992e-08
max ddh =  2.3389728891e-05
step = 18073  avg d_h= 0.129416130283
Number of iterations = 1 tolerance = 1.51188189288e-05
Number of iterations = 2 tolerance = 1.72677594248e-08
max ddh =  2.33885419616e-05
step = 18074  avg d_h= 0.129414111686
Number of iterations = 1 tolerance = 1.51193506681e-05
Number of iterations = 2 tolerance = 1.15875410694e-08
max ddh =  2.33873552362e-05
step = 18075  avg d_h= 0.129412093158
Number of iterations = 1 tolerance = 1.511987094e-05
Number of iterations = 2 tolerance = 8.45364580034e-09
max ddh =  2.33861688247e-05
step = 18076  avg d_h= 0.129410074699
Number of iterations = 1 tolerance = 1.51204275899e-05
Number of iterations = 2 tolerance = 1.0063580126e-08
max ddh =  2.33849827757e-05
step = 18077  avg d_h= 0.129408056309
Number of iterations = 1 tolerance = 1.51210677223e-05
Number of iterations = 2 tolerance = 1.55675538596e-08
max ddh =  2.33837974221e-05
step = 18078  avg d_h= 0.1294060379

Number of iterations = 1 tolerance = 1.51410615842e-05
Number of iterations = 2 tolerance = 1.21679578002e-09
max ddh =  2.33339409044e-05
step = 18120  avg d_h= 0.129321330943
Number of iterations = 1 tolerance = 1.51415926182e-05
Number of iterations = 2 tolerance = 1.13284428274e-09
max ddh =  2.33327551348e-05
step = 18121  avg d_h= 0.129319315592
Number of iterations = 1 tolerance = 1.51421186914e-05
Number of iterations = 2 tolerance = 1.11538730175e-09
max ddh =  2.3331569598e-05
step = 18122  avg d_h= 0.12931730031
Number of iterations = 1 tolerance = 1.51426595483e-05
Number of iterations = 2 tolerance = 1.09838894979e-09
max ddh =  2.33303843293e-05
step = 18123  avg d_h= 0.129315285097
Number of iterations = 1 tolerance = 1.51431503763e-05
Number of iterations = 2 tolerance = 1.08180491838e-09
max ddh =  2.33291990604e-05
step = 18124  avg d_h= 0.129313269953
Number of iterations = 1 tolerance = 1.51437019544e-05
Number of iterations = 2 tolerance = 1.06561502201e-09
max ddh

Number of iterations = 1 tolerance = 1.51675820477e-05
Number of iterations = 2 tolerance = 1.84673225496e-08
max ddh =  2.32784468642e-05
step = 18167  avg d_h= 0.12922668378
Number of iterations = 1 tolerance = 1.51683341577e-05
Number of iterations = 2 tolerance = 2.442594846e-08
max ddh =  2.32772739882e-05
step = 18168  avg d_h= 0.129224671656
Number of iterations = 1 tolerance = 1.51691793993e-05
Number of iterations = 2 tolerance = 3.44071313055e-08
max ddh =  2.32761021088e-05
step = 18169  avg d_h= 0.1292226596
Number of iterations = 1 tolerance = 1.51701625244e-05
Number of iterations = 2 tolerance = 2.75445242994e-08
max ddh =  2.32749313257e-05
step = 18170  avg d_h= 0.129220647613
Number of iterations = 1 tolerance = 1.51707509469e-05
Number of iterations = 2 tolerance = 4.86363512986e-08
max ddh =  2.32737599864e-05
step = 18171  avg d_h= 0.129218635694
Number of iterations = 1 tolerance = 1.5170820027e-05
Number of iterations = 2 tolerance = 3.35719593574e-08
max ddh =  

KeyboardInterrupt: 

In [6]:
figure(figsize=(20,12))
plot_links(mg, 'hydraulic__diameter', var_name='Hydraulic diameter ($m$)', pixels_per_node=15, lw=5)

In [8]:
mc = MeltCreep(mg, dt=10000.)
